In [2]:
!pip install scapy

In [3]:
filter_condition_MITM = "(eth_src == 'f0:18:98:5e:ff:9f' | eth_dst == 'f0:18:98:5e:ff:9f') & ((((ip_src == '192.168.0.16') & (ip_dst == '192.168.0.13')) | ((ip_src == '192.168.0.13') & (ip_dst == '192.168.0.16'))) & (icmp=='0') & tcp==1) | ((arp_src_mac == 'f0:18:98:5e:ff:9f') & ((arp_dst_mac == 'bc:1c:81:4b:ae:ba') | (arp_dst_mac == '48:4b:aa:2c:d8:f9')))"
filter_condition_DOS= "(tcp_flags_syn==1 & ip_dst == '192.168.0.24' & tcp_dport==19604 & tcp==1)"
filter_condition_Scanning="(((eth_src == 'f0:18:98:5e:ff:9f' & arp==1 & eth_dst == 'ff:ff:ff:ff:ff:ff') and framenumber < 13000) | (ip_src == '192.168.0.15' & ip_dst == '192.168.0.13' & ((tcp_flags_syn==1 & tcp_window==1024) or tcp_flag_reset == 1) ))"
filter_condition_Mirai_Botnet="ip_dst == '210.89.164.90'"
filter_condition_Normal=""

In [ ]:
from scapy.all import *
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
def build_dict(file,filter_condition,label,filename):
   
    
    #create empty record so we will have all features presented
    #even if they might exclude each other like UDP or TCP
    record1 = {}
    for feat in ['eth_dst','eth_src','eth_type','ip_version','ip_ihl','ip_tos','ip_len','ip_id','ip_flags','arp_src_ipv4','arp_dst_ipv4',
                 'arp_src_mac','arp_dst_mac','ip_frag','ip_ttl','ip_proto','ip_chksum','ip_src','ip_dst','udp_sport','udp_dport',
                 'udp_len','udp_chksum','icmp','tcp','tcp_sport','tcp_flags_syn','tcp_dport','tcp_seq','tcp_ack','tcp_dataofs','tcp_reserved',
                 'tcp_flags','tcp_window','tcp_chksum','tcp_urgptr','framenumber','arp','tcp_flag_reset','target']:
        record1[feat] = -1
    
    pcap = rdpcap(file)
    print(len(pcap))
    #filter_condition = "(eth_src == 'f0:18:98:5e:ff:9f' | eth_dst == 'f0:18:98:5e:ff:9f') & ((((ip_src == '192.168.0.16') & (ip_dst == '192.168.0.13')) | ((ip_src == '192.168.0.13') & (ip_dst == '192.168.0.16'))) & (icmp=='0') & tcp==1) | ((arp_src_mac == 'f0:18:98:5e:ff:9f') & ((arp_dst_mac == 'bc:1c:81:4b:ae:ba') | (arp_dst_mac == '48:4b:aa:2c:d8:f9')))"

    
    df = pd.DataFrame()
    CNT = [0]
    coun=0
    for pkt in pcap:
        
            record = record1
            record['target']=label
           
            if pkt.haslayer('frame_info'):
                # Extract the frame number from the frame_info attribute
                record['framenumber'] = pkt.frame_info.number
                
            if pkt.haslayer(Ether):
                record['eth_dst'] = pkt[Ether].dst
                record['eth_src'] = pkt[Ether].src
                record['eth_type'] = pkt[Ether].type
                
            if pkt.haslayer(IP):
                record['ip_version'] = pkt[IP].version
                record['ip_ihl'] = pkt[IP].ihl
                record['ip_tos'] = pkt[IP].tos
                record['ip_len'] = pkt[IP].len
                record['ip_id'] = pkt[IP].id
                record['ip_flags'] = pkt[IP].flags
                record['ip_frag'] = pkt[IP].frag
                record['ip_ttl'] = pkt[IP].ttl
                record['ip_proto'] = pkt[IP].proto
                record['ip_chksum'] = pkt[IP].chksum
                record['ip_src'] = pkt[IP].src
                record['ip_dst'] = pkt[IP].dst
                
                
            if pkt.haslayer(UDP):
                record['udp_sport'] = pkt[UDP].sport 
                record['udp_dport'] = pkt[UDP].dport
                record['udp_len'] = pkt[UDP].len
                record['udp_chksum'] = pkt[UDP].chksum
            
                
            if pkt.haslayer(ICMP):
                record['icmp'] = 1
            
            if pkt.haslayer(ARP):
                arp_layer = pkt[ARP]
                record['arp'] = 1
                record['arp_src_ipv4'] = arp_layer.psrc
                record['arp_dst_ipv4'] = arp_layer.pdst
                record['arp_src_mac'] = arp_layer.hwsrc
                record['arp_dst_mac'] = arp_layer.hwdst
                
                
            if pkt.haslayer(TCP):
                if pkt[TCP].haslayer(Raw):
                    record['tcp']=1
                record['tcp_sport'] = pkt[TCP].sport
                record['tcp_dport'] = pkt[TCP].dport
                record['tcp_seq'] = pkt[TCP].seq
                record['tcp_ack'] = pkt[TCP].ack
                record['tcp_dataofs'] = pkt[TCP].dataofs
                record['tcp_reserved'] = pkt[TCP].reserved
                record['tcp_flags'] = pkt[TCP].flags
                if(pkt[TCP].flags.S):
                    record['tcp_flags_syn'] =1
                record['tcp_window'] = pkt[TCP].window
                record['tcp_chksum'] = pkt[TCP].chksum
                record['tcp_urgptr'] = pkt[TCP].urgptr
                if(pkt[TCP].flags.R):
                    record['tcp_flag_reset'] =1
            '''new = pd.DataFrame()
            new = new.append(record,ignore_index=True)
            result=(new.query(filter_condition))
            
            if(len(result)>0):
                coun=coun+1
                print(coun)
                df = df.append(record,ignore_index=True)'''
            
            df = df.append(record,ignore_index=True)
                

    df.to_csv(filename)
    
    return df

packets = build_dict('Dataset/benign-dec.pcap',filter_condition_Normal,"BENIGN","CSVFile/BENIGN.csv")

137396


In [ ]:
packets1 = packets.head(10000)
packets1.to_csv("CSVFile/BENIGN.csv")

In [ ]:
packets.to_csv("BENIGN.csv")